<a href="https://colab.research.google.com/github/EduardoAve/Labour-well-being/blob/main/Data_preparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Preparación de datos sobre Educadores en República Checa y Austria

##Introducción

Este informe documenta el proceso de limpieza y preparación de un conjunto de datos que recopila información sobre educadores en instituciones de educación superior en la República Checa y Austria. El dataset original consta de 129 columnas, cada una representando una pregunta o respuesta de una encuesta aplicada a los participantes. Sin embargo, en su estado inicial, los datos presentaban diversos problemas que dificultaban su análisis directo.

Nuestro objetivo fue transformar y estructurar el dataset de manera adecuada, asegurando la coherencia de sus variables y generando nuevas columnas derivadas de otras respuestas, lo que permitirá describir mejor ciertos aspectos clave de los educadores en el estudio. Para ello, se llevaron a cabo las siguientes acciones:

- Cálculo de nuevas variables mediante promedios y otras transformaciones, generando columnas objetivo que no estaban presentes en la versión original del dataset.
- Renombrado de columnas, asegurando nombres claros y uniformes.
- Corrección de tipos de datos, adaptando cada variable a su formato adecuado (numérico, categórico, etc.).
- Manejo de valores nulos e inconsistencias, para evitar sesgos y errores en el análisis.

Con estas modificaciones, el dataset ahora se encuentra estructurado y listo para ser utilizado en análisis posteriores.



In [170]:
import pandas as pd
import numpy as np

In [171]:
df = pd.read_excel('/content/drive/MyDrive/labour well being/Data prepatartion/Dataset_Labour_Wellbeing.xlsx')

Se presenta una pequeña visualización del conjunto de datos inicial

In [172]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Columns: 129 entries, Country to What bothers you most about your work at a higher education institution at the moment? What do you like most?
dtypes: float64(120), int64(3), object(6)
memory usage: 2.7+ MB


In [173]:
# Mostrar los nombres de todas las columnas
print(df.columns)

Index(['Country', 'Version', '1. Gender:', '2. Age (in years):', 'Unnamed: 4',
       '3. Nationality:', '4. Current marital (partnership) status:',
       '5. Do you currently care for underage children or dependent relatives?',
       '6. The type of higher education insitution where you primarily work:',
       '7. Subject area of the faculty (higher education institution) where you primarily work:',
       ...
       '36. I tend to overwork.',
       '37. I don’t consider my work to be finished until I am completely satisfied with the result. ',
       '38. My thoughts revolve around work almost exclusively.',
       '39. When I’m unsuccessful at work, it makes me feel very down.',
       '40. If I don’t succeed at something, that just makes me all the more determined.',
       '41. I can be calm and collected in almost all situations. ',
       '42. My life up till now has been characterized by professional success. ',
       '43. By and large, I am happy and content. ',
       '4

In [174]:
# Renombrar la columna desconocida
df = df.rename(columns={'Unnamed: 4': 'Unknown_Column'})


#Se crean nuevas columnas


In [175]:
# Función para invertir valores en una escala de 1 a 5
def invert_scale(series):
    return series.apply(lambda x: 6 - x if pd.notna(x) else x)


# Crear las nuevas columnas calculadas
df['Academic_resources'] = df.iloc[:, 36:42].mean(axis=1)

df['Performance_pressure'] = df.iloc[:, 42]

df['Perceived_autonomy'] = (
    df.iloc[:, 43] +
    invert_scale(df.iloc[:, 44]) +
    invert_scale(df.iloc[:, 45]) +
    df.iloc[:, 46] +
    df.iloc[:, 47] +
    invert_scale(df.iloc[:, 48])
) / 6

df['Quality_of_leadership'] = df.iloc[:, 49:53].mean(axis=1)

df['Sense_of_community'] = df.iloc[:, 53:56].mean(axis=1)

df['Job_satisfaction'] = df.iloc[:, 56:61].mean(axis=1)

df['Burnout'] = df.iloc[:, 80:84].mean(axis=1)

# Eliminar las columnas que ya no se usarán
cols_to_drop = (
    list(range(36, 42)) +  # Academic resources originales
    [42] +  # Performance pressure original
    list(range(43, 49)) +  # Perceived autonomy originales
    list(range(49, 53)) +  # Quality of leadership originales
    list(range(53, 56)) +  # Sense of community originales
    list(range(56, 61)) +  # Job satisfaction originales
    list(range(80, 84)) +  # Burnout originales
    list(range(61, 80)) +  # Work motivation
    list(range(84, 106)) +  # Vulnerability to burnout
    list(range(106, 130))  # Columnas adicionales que mencionaste (107-129)
)

df.drop(df.columns[cols_to_drop], axis=1, inplace=True)



In [176]:
#Visualizamos como quedaron nuestras nuevas columnas calculadas
df.head()

,Country,Version,1. Gender:,2. Age (in years):,Unknown_Column,3. Nationality:,4. Current marital (partnership) status:,5. Do you currently care for underage children or dependent relatives?,6. The type of higher education insitution where you primarily work:,7. Subject area of the faculty (higher education institution) where you primarily work:,...,"1. Teaching (classroom instruction, preparation of instructional materials and lesson plans, advising students, reading and evaluating student work, examination management, etc.)","2. Research (reading literature, designing and conducting experiments, collecting and analysing data, writing articles or other scientific texts, etc.)","3. Activities related to externally funded research projects (searching for information on available funding sources, preparation of grant applications and project reports, project management and administration, etc.)","4. Organisational and administrative activities (organising and attending meetings, dealing with tasks and documents not directly related to teaching, research, or externally funded research projects, etc.)",Performance_pressure,Perceived_autonomy,Quality_of_leadership,Sense_of_community,Job_satisfaction,Burnout
0,2,4,2.0,28.0,2.0,Spanish,3.0,1.0,1.0,1.0,...,5.0,29.0,1.0,15,1.0,1.833333,3.00,2.666667,2.2,4.00
1,2,4,1.0,33.0,2.0,German,2.0,1.0,1.0,3.0,...,5.0,30.0,5.0,10,5.0,1.500000,2.00,1.666667,2.4,4.00
2,2,4,2.0,32.0,2.0,Kosovan,3.0,1.0,NaN,5.0,...,NaN,NaN,NaN,NaN,4.0,4.666667,5.00,5.000000,5.0,4.25
3,2,4,NaN,30.0,2.0,Spain,2.0,1.0,1.0,5.0,...,20.0,10.0,NaN,10,1.0,3.833333,2.00,3.000000,2.8,3.25
4,2,4,1.0,39.0,2.0,Italian,2.0,2.0,1.0,5.0,...,20.0,20.0,10.0,3,5.0,3.166667,3.75,3.000000,2.6,4.50


#La columna 26 es un duplicado de la 25, se eliminará.


In [177]:
# Eliminar la primera columna "14.1" (con el índice 24)
df = df.drop(df.columns[24], axis=1)

#A continuación, ponemos nombres más representativos para las columnas y que permitan un trabajo más cómodo:

In [178]:
# Diccionario de renombrado
rename_dict = {
    "Country": "Country",
    "Version": "Version",
    "1. Gender:": "Gender",
    "2. Age (in years):": "Age",
    "Unknown_Column": "Unknown_Column",
    "3. Nationality:": "Nationality",
    "4. Current marital (partnership) status:": "Marital_Status",
    "5. Do you currently care for underage children or dependent relatives?": "Care_Responsibilities",
    "6. The type of higher education insitution where you primarily work:": "HEI_Type",
    "7. Subject area of the faculty (higher education institution) where you primarily work:": "Faculty_Subject_Area",
    "8. Duration of your current employment contract at the higher education institution where you primarily work:": "Employment_Contract_Duration",
    "9. Extent of employment in higher education (in hours/week, aggregated for all higher education institutions where you work):": "HEI_Employment_Hours",
    "10. Actual average weekly working hours in higher education (in a typical semester week):": "HEI_Actual_Weekly_Hours",
    "Effort (less, more, equal)": "Effort_Level",
    "Effort [%]": "Effort_Percentage",
    "Income CZK": "Income_CZK",
    "Income EUR": "Income_EUR",
    "Income EURO": "Income_EURO",
    "Euro Adj.": "Euro_Adjusted",
    "Salary/hour": "Salary_per_Hour",
    "Salary effort/hour": "Salary_Effort_per_Hour",
    "12. Do you hold a leadership position at a higher education institution?": "Leadership_Position",
    "13. How influential are you in helping to shape key academic policies at your institution at the level of department or similar unit?": "Policy_Influence",
    "14. Do you currently have another (paid) job outside higher education?": "Other_Paid_Job",
    "14.1. Actual average weekly working hours outside higher education (in a typical semester week):": "Other_Job_Weekly_Hours",
    "14.1. Actual average weekly working hours outside higher education (in a typical semester week): .1": "Other_Job_Weekly_Hours_1",
    "Academic/Non-academic": "Academic_or_Non_Academic",
    "CZ_15. Your current position at the higher education institution, where you primarily work: ": "Current_Position_CZ",
    "AT_15. Your current position at the higher education institution, where you primarily work: ": "Current_Position_AT",
    "16. Please choose the category that best fits your job description:": "Job_Category",
    "17. The highest level of education attained:": "Highest_Education_Level",
    "18. Total length of your career in Czech higher education in years:": "Career_Length_CZ",
    "1. Teaching (classroom instruction, preparation of instructional materials and lesson plans, advising students, reading and evaluating student work, examination management, etc.)": "Teaching_Hours",
    "2. Research (reading literature, designing and conducting experiments, collecting and analysing data, writing articles or other scientific texts, etc.)": "Research_Hours",
    "3. Activities related to externally funded research projects (searching for information on available funding sources, preparation of grant applications and project reports, project management and administration, etc.)": "Funded_Research_Activities",
    "4. Organisational and administrative activities (organising and attending meetings, dealing with tasks and documents not directly related to teaching, research, or externally funded research projects, etc.)": "Administrative_Activities",
    "Performance_pressure": "Performance_Pressure",
    "Perceived_autonomy": "Perceived_Autonomy",
    "Quality_of_leadership": "Quality_of_Leadership",
    "Sense_of_community": "Sense_of_Community",
    "Job_satisfaction": "Job_Satisfaction",
    "Burnout": "Burnout"
}


# Renombrar columnas
df = df.rename(columns=rename_dict)

#Los nuevos nombres de las columnas quedaron de la siguiente manera:

In [179]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country                       2748 non-null   int64  
 1   Version                       2748 non-null   int64  
 2   Gender                        2739 non-null   float64
 3   Age                           2699 non-null   float64
 4   Unknown_Column                2707 non-null   float64
 5   Nationality                   270 non-null    object 
 6   Marital_Status                2720 non-null   float64
 7   Care_Responsibilities         2729 non-null   float64
 8   HEI_Type                      2744 non-null   float64
 9   Faculty_Subject_Area          2720 non-null   float64
 10  Employment_Contract_Duration  2745 non-null   float64
 11  HEI_Employment_Hours          2712 non-null   object 
 12  HEI_Actual_Weekly_Hours       2558 non-null   float64
 13  Eff

Las columnas de "HEI_Employment_Hours", "Income_EUR", "Career_Length_CZ", y "Administrative_Activities" no deberían ser objetos, hacemos el cambio de tipo de dato.  

In [180]:
df["HEI_Employment_Hours"] = pd.to_numeric(df["HEI_Employment_Hours"], errors="coerce")
df["Income_EUR"] = pd.to_numeric(df["Income_EUR"], errors="coerce")
df["Career_Length_CZ"] = pd.to_numeric(df["Career_Length_CZ"], errors="coerce")
df["Administrative_Activities"] = pd.to_numeric(df["Administrative_Activities"], errors="coerce")


In [181]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2748 entries, 0 to 2747
Data columns (total 41 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Country                       2748 non-null   int64  
 1   Version                       2748 non-null   int64  
 2   Gender                        2739 non-null   float64
 3   Age                           2699 non-null   float64
 4   Unknown_Column                2707 non-null   float64
 5   Nationality                   270 non-null    object 
 6   Marital_Status                2720 non-null   float64
 7   Care_Responsibilities         2729 non-null   float64
 8   HEI_Type                      2744 non-null   float64
 9   Faculty_Subject_Area          2720 non-null   float64
 10  Employment_Contract_Duration  2745 non-null   float64
 11  HEI_Employment_Hours          2711 non-null   float64
 12  HEI_Actual_Weekly_Hours       2558 non-null   float64
 13  Eff

#Valores nulos de cada columna:

In [182]:
df.isnull().sum()


,0
Country,0
Version,0
Gender,9
Age,49
Unknown_Column,41
Nationality,2478
Marital_Status,28
Care_Responsibilities,19
HEI_Type,4
Faculty_Subject_Area,28


#Se guarda el nuevo conjunto de datos en un excel y CSV

In [183]:
df.to_csv("labour_well_being_data.csv", index=False, encoding="utf-8")

df.to_excel("labour_well_being_data.xlsx", index=False, engine="openpyxl")
